##### upload csv file

In [1]:
# import numpy and pandas
import numpy as np
import pandas as pd
from sklearn import preprocessing

# used for dates
import datetime
from datetime import datetime, date

# Set some pandas options controlling output format
pd.set_option('display.notebook_repr_html', False)
pd.set_option('display.max_columns', 8)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 90)

# bring in matplotlib for graphics
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# read output #2 (new column for memeber tenure tier)
root = 'data'
msft = pd.read_csv(root + '/' +'3.product_label_binarizer.csv')
#msft[:15]

In [3]:
df = msft.copy()

In [4]:
df.shape

(21343, 114)

In [5]:
spike_cols = [col for col in df.columns if 'Tier' in col ]
print(spike_cols, "\n")

['Member_Tenure_Tier'] 



In [6]:
spike_cols = [col for col in df.columns if 'Flag' in col ]
print(spike_cols, "\n")
spike_cols = [col for col in df.columns if 'FSV' in col ]
print(spike_cols, "\n")
spike_cols = [col for col in df.columns if 'TRV' in col ]
print(spike_cols, "\n")

['Member_Flag', 'FSV_CMSI_Flag', 'FSV_Credit_Card_Flag', 'FSV_Deposit_Program_Flag', 'FSV_Home_Equity_Flag', 'FSV_ID_Theft_Flag', 'FSV_Mortgage_Flag', 'INS_Client_Flag', 'TRV_Globalware_Flag', 'Bad_Address_Flag', 'New_Mover_Flag', 'Member_Match_Flag'] 

['FSV_CMSI_Flag', 'FSV_Credit_Card_Flag', 'FSV_Deposit_Program_Flag', 'FSV_Home_Equity_Flag', 'FSV_ID_Theft_Flag', 'FSV_Mortgage_Flag'] 

['TRV_Globalware_Flag'] 



In [7]:
a =df[['Individual_Key','FSV_CMSI_Flag', 'FSV_Credit_Card_Flag', 'FSV_Deposit_Program_Flag', 
 'FSV_Home_Equity_Flag', 'FSV_ID_Theft_Flag', 'FSV_Mortgage_Flag','TRV_Globalware_Flag']]
df_temp = a.drop_duplicates(subset=None, keep='first', inplace=False)
print(df.shape)
print(a.shape)
print(df_temp.shape)
df_temp[:10]
#df['Product_Count']

(21343, 114)
(21343, 8)
(11902, 8)


    Individual_Key  FSV_CMSI_Flag  FSV_Credit_Card_Flag  FSV_Deposit_Program_Flag  \
0       52211550.0              0                     1                         0   
9        1606764.0              0                     0                         0   
10       2766867.0              0                     0                         0   
13       2766869.0              0                     0                         0   
14       2766868.0              0                     0                         0   
19      13746947.0              0                     0                         0   
20       1788453.0              0                     0                         0   
21       1788452.0              0                     0                         0   
25       1788455.0              0                     0                         0   
26      14243585.0              0                     0                         0   

    FSV_Home_Equity_Flag  FSV_ID_Theft_Flag  FSV_Mortgage_Flag  

In [8]:
df_sum = df_temp.set_index('Individual_Key')

#Total sum per row: 
df_sum['Product_Count'] = df_sum.sum(axis=1)
df_sum.loc[52211550.0]

FSV_CMSI_Flag               0
FSV_Credit_Card_Flag        1
FSV_Deposit_Program_Flag    0
FSV_Home_Equity_Flag        0
FSV_ID_Theft_Flag           0
FSV_Mortgage_Flag           0
TRV_Globalware_Flag         0
Product_Count               1
Name: 52211550.0, dtype: int64

In [9]:
df_temp = df_sum.reset_index().copy()
df_temp.columns
df_total = df_temp[['Individual_Key', 'Product_Count']].copy()

In [10]:
# find the column index
df.columns.get_loc('TRV_Globalware_Flag')

14

In [11]:
# adding the product count into the orginal dataset
df_result = pd.merge(df, df_total, on='Individual_Key')
print(df_result.shape)
print(df_result.columns)

(21343, 115)
Index(['Individual_Key', 'Household_Key', 'Member_Flag', 'City', 'State_-_Grouped',
       'ZIP5', 'ZIP9', 'FSV_CMSI_Flag', 'FSV_Credit_Card_Flag',
       'FSV_Deposit_Program_Flag',
       ...
       'SVC_Facility_Type', 'Total_Cost', 'Tow_Destination_Latitude',
       'Tow_Destination_Longitude', 'Tow_Destination_Name', 'Was_Duplicated',
       'Was_Towed_To_AAR_Referral', 'Member_Tenure_Decile', 'Member_Tenure_Tier',
       'Product_Count'],
      dtype='object', length=115)


In [12]:
# add new column to the 1st position
df_result.insert(15, 'Total_Products', df_result['Product_Count'])
df_result['Total_Products']
del df_result['Product_Count']

In [13]:
# a client has max 4 products
print(sorted(df_result['Total_Products'].unique()))

[0, 1, 2, 3, 4]


In [14]:
print("# of Right_Individual_Key not same as Right_Individual_Key: ", \
      df_result[df_result['Right_Individual_Key'] != df_result['Individual_Key']].shape[0])


# of Right_Individual_Key not same as Right_Individual_Key:  0


In [15]:
# delete the column
del df_result['Right_Individual_Key']

In [16]:
spike_cols = [col for col in df.columns if 'Member' in col ]
print(spike_cols, "\n")

['Member_Flag', 'ERS_Member_Cost_Year_1', 'ERS_Member_Cost_Year_2', 'ERS_Member_Cost_Year_3', 'Member_Key', 'Member_Map_Location', 'Member_Number_Associate_ID', 'Member_Phone_Type', 'Member_Status', 'Member_Tenure_Years', 'Member_Type', 'Membership_ID', 'Member_Match_Flag', 'Member_Number_and_Associate_ID', 'Member_Tenure_Decile', 'Member_Tenure_Tier'] 



In [17]:
print("unique member flag:", df_result['Member_Flag'].unique())
print("has nan?:", df_result[df_result['Member_Flag'].isnull()].shape[0])

unique member flag: ['Y']
has nan?: 0


In [18]:
# all the clients are AAA member - omit it
del df_result['Member_Flag']

In [19]:
df_result.columns

Index(['Individual_Key', 'Household_Key', 'City', 'State_-_Grouped', 'ZIP5', 'ZIP9',
       'FSV_CMSI_Flag', 'FSV_Credit_Card_Flag', 'FSV_Deposit_Program_Flag',
       'FSV_Home_Equity_Flag',
       ...
       'SVC_Facility_Name', 'SVC_Facility_Type', 'Total_Cost',
       'Tow_Destination_Latitude', 'Tow_Destination_Longitude', 'Tow_Destination_Name',
       'Was_Duplicated', 'Was_Towed_To_AAR_Referral', 'Member_Tenure_Decile',
       'Member_Tenure_Tier'],
      dtype='object', length=113)

In [20]:
z1 = df_result['Occupation_Group'].dropna()
z1.head()

19                     Retired - Known
21    Office Administration - Inferred
22    Office Administration - Inferred
23    Office Administration - Inferred
24    Office Administration - Inferred
Name: Occupation_Group, dtype: object

In [21]:
# # write the output to data sub-directory
root = 'data'
df_result.to_csv( root + '/' +'4.total_products_cleanup.csv',index=False)